<a href="https://colab.research.google.com/github/samarrkilfaden-droid/advanced_ai_exercises/blob/main/task2(wgan)_Day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
# ---------------------------
# Data  (CIFAR-10 32×32)
# ---------------------------
transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])  # OK for Tanh normalizes the ranges from -1 to 1
])
ds = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform)
loader = DataLoader(ds, batch_size=64, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)

100%|██████████| 170M/170M [00:03<00:00, 48.8MB/s]


In [4]:
# ---------------------------
# Hyperparams
# ---------------------------
z_dim = 128
g_lr  = 2e-4
d_lr  = 2e-4
n_critic = 5                     # critic should be 5 not 1 in wgans
lambda_gp = 10.0

In [6]:
class Critic(nn.Module):
    def __init__(self, ch=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, ch,   4, 2, 1),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(ch, ch*2, 4, 2, 1),
            nn.InstanceNorm2d(ch*2, affine=True),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(ch*2, ch*4, 4, 2, 1),
            nn.InstanceNorm2d(ch*4, affine=True),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(ch*4, 1, 4, 1, 0),
            #nn.Sigmoid()                 # remove sigmoid
        )
    def forward(self, x):
       return self.net(x).view(x.size(0))

class Gen(nn.Module):
    def __init__(self, z=128, ch=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z,   ch*4, 4, 1, 0, bias=False), nn.BatchNorm2d(ch*4), nn.ReLU(True),
            nn.ConvTranspose2d(ch*4, ch*2, 4, 2, 1, bias=False), nn.BatchNorm2d(ch*2), nn.ReLU(True),
            nn.ConvTranspose2d(ch*2, ch,   4, 2, 1, bias=False), nn.BatchNorm2d(ch),   nn.ReLU(True),
            nn.ConvTranspose2d(ch,   3,    4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, z):
      return self.net(z.view(z.size(0), z.size(1), 1, 1))

D = Critic().to(device)
G = Gen(z_dim).to(device)

In [7]:
# ---------------------------
# Loss placeholders & Optimizers (WRONG for WGAN… intentionally)
# ---------------------------
# Loss (no BCE, just mean differences)
def d_loss(real_scores, fake_scores):
    return torch.mean(fake_scores) - torch.mean(real_scores)

def g_loss(fake_scores):
    return -torch.mean(fake_scores)

# Optimizers
optG = torch.optim.RMSprop(G.parameters(), lr=5e-5) # recommended for WGAN
optD = torch.optim.RMSprop(D.parameters(), lr=5e-5) # recommended for WGAN

In [9]:
# ---------------------------
# Broken gradient penalty
# ---------------------------
def gradient_penalty(Dnet, real, fake):
    b = real.size(0)
    eps = torch.randn(b,1,1,1 ,device=real.device)             # BUG
    x_hat = eps*real + (1-eps)*fake
    # BUG
    d_hat = Dnet(x_hat)
    grads = torch.autograd.grad(d_hat.sum(), x_hat, retain_graph=True)[0]    # WRONG
    gp = lambda_gp * (grads.view(b, -1).norm(dim=1) - 1.0).mean()            # WRONG
    return gp

In [10]:
# ---------------------------
# Training loop (intentionally wrong)
# ---------------------------
for step, (real, _) in enumerate(loader):
    real = real.to(device)
    b = real.size(0)

    # -- Critic updates --
    for _ in range(n_critic):                    # BUG
        z = torch.randn(b, z_dim, device=device)
        fake = G(z).detach()

        # WRONG
        d_real = D(real)
        d_fake = D(fake)

        lossD = torch.mean(fake) - torch.mean(real) + \
                gradient_penalty(D, real, fake)

        lossD.backward()
        optD.step()
        optD.zero_grad()


        # BUG
        #for p in D.parameters():
            #p.data.clamp_(-0.01, 0.01)

    # -- Generator update --
    z = torch.randn(b, z_dim, device=device)
    fake = G(z)
    # WRONG
    lossG = -torch.mean(fake)   # BUG
    lossG.backward()
    optG.step()
    optG.zero_grad()                        # BUG

    if step > 10:   # keep the broken demo short
        break

print("Your task: fix all bugs until the WGAN-GP training runs stably.")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


RuntimeError: One of the differentiated Tensors does not require grad

In [ ]:
# ---------------------------
# Setup
# ---------------------------
D.train()
G.train()

lambda_gp = 10.0         # gradient penalty coefficient (standard)
n_critic  = 5            # critic steps per generator step (standard)
g_lr = d_lr = 5e-5

optG = torch.optim.RMSprop(G.parameters(), lr=g_lr)  # WGAN (vanilla) uses RMSprop; GP works fine with it too
optD = torch.optim.RMSprop(D.parameters(), lr=d_lr)

# If your G is ConvTranspose2d-based (DCGAN-style), z should be [B, z_dim, 1, 1]
def sample_z(batch, z_dim, device):
    return torch.randn(batch, z_dim, 1, 1, device=device)  # <-- change to (batch, z_dim) if your G expects flat input

# ---------------------------
# Losses (WGAN)
# ---------------------------
def d_loss(real_scores, fake_scores):
    return fake_scores.mean() - real_scores.mean()

def g_loss(fake_scores):
    return -fake_scores.mean()

# ---------------------------
# Gradient Penalty (WGAN-GP)
# ---------------------------
def gradient_penalty(Dnet, real, fake, lambda_gp=10.0):
    b = real.size(0)
    # ε ~ U[0,1], broadcast across non-batch dims
    eps = torch.rand(b, 1, 1, 1, device=real.device, dtype=real.dtype)
    x_hat = eps * real + (1.0 - eps) * fake
    x_hat.requires_grad_(True)

    d_hat = Dnet(x_hat)                     # [B] or [B,1,...]
    d_hat = d_hat.view(b)                   # flatten to [B] if needed

    grads = torch.autograd.grad(
        outputs=d_hat.sum(),                # scalar
        inputs=x_hat,
        create_graph=True,                  # retain graph for higher-order grad
        retain_graph=True,
        only_inputs=True
    )[0]                                    # [B, C, H, W]

    grad_norm = grads.view(b, -1).norm(2, dim=1)     # 2
    gp = lambda_gp * ((grad_norm - 1.0) ** 2).mean() # ( - 1)^2
    return gp

# ---------------------------
# Training loop (fixed)
# ---------------------------
for step, (real, _) in enumerate(loader):
    real = real.to(device)
    b = real.size(0)

    # ---- Critic updates ----
    for _ in range(n_critic):
        # (1) Sample noise and make fake (detach so D update doesn't backprop to G)
        z = sample_z(b, z_dim, device)
        with torch.no_grad():
            fake = G(z)

        # (2) Critic scores
        real_scores = D(real).view(b)
        fake_scores = D(fake).view(b)

        # (3) WGAN loss + GP
        gp = gradient_penalty(D, real, fake, lambda_gp=lambda_gp)
        lossD = d_loss(real_scores, fake_scores) + gp

        # (4) Optimize critic
        optD.zero_grad(set_to_none=True)
        lossD.backward()
        optD.step()

    # ---- Generator update ----
    z = sample_z(b, z_dim, device)
    fake = G(z)
    fake_scores = D(fake).view(b)
    lossG = g_loss(fake_scores)

    optG.zero_grad(set_to_none=True)
    lossG.backward()
    optG.step()

    # (Optional) short-run break for debugging
    # if step > 10: break